In [25]:
#NOTE:
#there is no 04/2020 for the bike data

In [26]:
def line(char="=", length=50):
    return char * length

In [18]:
import zipfile
import pandas as pd
import glob
import gc


In [19]:
def load_and_aggregate_daily_stats(data_folder="data2"):
    zip_files = sorted(glob.glob(f"{data_folder}/*.zip"))
    print(line())
    print(f"Processing {len(zip_files)} files...")
    print(line())
    daily_sums = {}
    for i, zf in enumerate(zip_files, start=1):
        print(f"[{i}/{len(zip_files)}] {zf}")
        
        with zipfile.ZipFile(zf, 'r') as z:
            csv_name = z.namelist()[0]
            with z.open(csv_name) as f:
                for chunk in pd.read_csv(f, chunksize=50000, low_memory=False):
                    # Find time column
                    time_col = next((c for c in ['start_time', 'started_at', 'Start Time'] if c in chunk.columns), None)
                    if not time_col:
                        continue
                    # Find duration column
                    dur_col = next((c for c in ['duration_sec', 'Duration', 'tripduration']if c in chunk.columns), None)
                    
                    # Clean dates
                    chunk[time_col] = pd.to_datetime(chunk[time_col], errors='coerce')
                    chunk = chunk.dropna(subset=[time_col])
                    chunk['date'] = chunk[time_col].dt.date
            
                    # Aggregate by date
                    for date, grp in chunk.groupby('date'):
                        if date not in daily_sums:
                            daily_sums[date] = {'dur': 0, 'trips': 0, 'bike_share': 0, 'subs': 0}
                        
                        if dur_col:
                            daily_sums[date]['dur'] += grp[dur_col].sum()
                        daily_sums[date]['trips'] += len(grp)
                        if 'bike_share_for_all_trip' in grp.columns:
                            daily_sums[date]['bike_share'] += (grp['bike_share_for_all_trip'] == 'Yes').sum()
                        if 'user_type' in grp.columns:
                            daily_sums[date]['subs'] += (grp['user_type'] == 'Subscriber').sum()
        if i % 10 == 0:
            gc.collect()
    
    rows = []
    for date, s in daily_sums.items():
        rows.append({
            'date': date,
            'avg_duration_sec': s['dur'] / s['trips'],
            'pct_bike_share_yes': (s['bike_share'] / s['trips']) * 100,
            'pct_subscriber': (s['subs'] / s['trips']) * 100,
            'total_trips': s['trips']
        })
    
    df = pd.DataFrame(rows).sort_values('date').set_index('date')
    print(f"\nDone: {len(df)} days, {df['total_trips'].sum():,.0f} trips")
    
    return df

In [20]:
daily_stats = load_and_aggregate_daily_stats()
daily_stats.to_csv('daily_stats.csv')

Processing 83 files...
[1/83] data2/201801-fordgobike-tripdata.csv.zip
[2/83] data2/201802-fordgobike-tripdata.csv.zip
[3/83] data2/201803-fordgobike-tripdata.csv.zip
[4/83] data2/201804-fordgobike-tripdata.csv.zip
[5/83] data2/201805-fordgobike-tripdata.csv.zip
[6/83] data2/201806-fordgobike-tripdata.csv.zip
[7/83] data2/201807-fordgobike-tripdata.csv.zip
[8/83] data2/201808-fordgobike-tripdata.csv.zip
[9/83] data2/201809-fordgobike-tripdata.csv.zip
[10/83] data2/201810-fordgobike-tripdata.csv.zip
[11/83] data2/201811-fordgobike-tripdata.csv.zip
[12/83] data2/201812-fordgobike-tripdata.csv.zip
[13/83] data2/201901-fordgobike-tripdata.csv.zip
[14/83] data2/201902-fordgobike-tripdata.csv.zip
[15/83] data2/201903-fordgobike-tripdata.csv.zip
[16/83] data2/201904-fordgobike-tripdata.csv.zip
[17/83] data2/201905-baywheels-tripdata.csv.zip
[18/83] data2/201906-baywheels-tripdata.csv.zip


/tmp/ipykernel_171/1439153565.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk[time_col] = pd.to_datetime(chunk[time_col], errors='coerce')


[19/83] data2/201907-baywheels-tripdata.csv.zip
[20/83] data2/201908-baywheels-tripdata.csv.zip
[21/83] data2/201909-baywheels-tripdata.csv.zip
[22/83] data2/201910-baywheels-tripdata.csv.zip
[23/83] data2/201911-baywheels-tripdata.csv.zip
[24/83] data2/201912-baywheels-tripdata.csv.zip
[25/83] data2/202001-baywheels-tripdata.csv.zip
[26/83] data2/202002-baywheels-tripdata.csv.zip
[27/83] data2/202003-baywheels-tripdata.csv.zip
[28/83] data2/202005-baywheels-tripdata.csv.zip
[29/83] data2/202006-baywheels-tripdata.csv.zip
[30/83] data2/202007-baywheels-tripdata.csv.zip
[31/83] data2/202008-baywheels-tripdata.csv.zip
[32/83] data2/202009-baywheels-tripdata.csv.zip
[33/83] data2/202010-baywheels-tripdata.csv.zip
[34/83] data2/202011-baywheels-tripdata.csv.zip
[35/83] data2/202012-baywheels-tripdata.csv.zip
[36/83] data2/202101-baywheels-tripdata.csv.zip
[37/83] data2/202102-baywheels-tripdata.csv.zip
[38/83] data2/202103-baywheels-tripdata.csv.zip
[39/83] data2/202104-baywheels-tripdata.

In [24]:
# Load any NOAA CSV of type
# https://www.ncei.noaa.gov/access/search/data-search/daily-summaries?bbox=38.075,-122.715,37.485,-122.125&startDate=2018-01-01T00:00:00&endDate=2024-12-31T23:59:59&pageNum=1

df = pd.read_csv("USW00023272.csv")

df['DATE'] = pd.to_datetime(df['DATE'])

mask = (df['DATE'] >= '2018-01-01') & (df['DATE'] <= '2024-12-31')
df_filtered = df[mask]

print(f"Original rows: {len(df):,}")
print(f"Filtered rows: {len(df_filtered):,}")
print(f"Date range: {df_filtered['DATE'].min()} to {df_filtered['DATE'].max()}")

output_file = "weather_2018_2024.csv"
df_filtered.to_csv(output_file, index=False)
print(f"✓ Exported to: {output_file}")

Original rows: 38,326
Filtered rows: 2,557
Date range: 2018-01-01 00:00:00 to 2024-12-31 00:00:00
✓ Exported to: weather_2018_2024.csv


/tmp/ipykernel_171/3037975613.py:4: DtypeWarning: Columns (17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("USW00023272.csv")
